# This notebook is used to compare the same sample (from the batch to be corrected) before and after correction, which is corrected by the same sample (from another batch used for correction).  

In [ ]:
source("mass_cytometry_functions.R")
library(caret)
library(emdist)
library(egg)
library(glue)

In [ ]:
emd_for_each_marker_for_two_fcs<- function(path_needed1, path_needed2, transform, sampling_number_emd, needed_antibody){
        dataframe_res1<- fcs_to_dataframe(path_needed1, 
                        needed_antibody,
                        transform)
        dataframe_res2<- fcs_to_dataframe(path_needed2, 
                        needed_antibody,
                        transform)
        distances<- c()
        for (column_needed in c(1:dim(dataframe_res2)[2])){
            A<- sample(dataframe_res1[ ,column_needed], sampling_number_emd, replace = TRUE)
            A<- matrix(A)
            B<- sample(dataframe_res2[ ,column_needed], sampling_number_emd, replace = TRUE)
            B<- matrix(B)
            distance<- (suppressWarnings(emd2d(A,B,dist="euclidean"))/sampling_number_emd)*100
            distances<- append(distances, distance)
        }
        results<- data.frame(column_names = colnames(dataframe_res2), distance = distances)
        return(results)
}




emd_for_each_marker_for_two_dataframe<- function(dataframe_res1, dataframe_res2, sampling_number_emd, needed_antibody){

        distances<- c()
        for (column_needed in c(1:dim(dataframe_res2)[2])){
            A<- sample(dataframe_res1[ ,column_needed], sampling_number_emd, replace = TRUE)
            A<- matrix(A)
            B<- sample(dataframe_res2[ ,column_needed], sampling_number_emd, replace = TRUE)
            B<- matrix(B)
            distance<- (suppressWarnings(emd2d(A,B,dist="euclidean"))/sampling_number_emd)*100
            distances<- append(distances, distance)
        }
        results<- data.frame(column_names = colnames(dataframe_res2), distance = distances)
        return(results)
}



histogram_of_each_column_for_three_dataframes<- function(dataframe1, dataframe2, dataframe3, title, xlab, bin, sampling_number, main_title){
   #' given dataframe
   #' given title name
   #' given xlab value
   #' give bin size
   #' output histogram of all columns
 dataframe1 <- dataframe1[sample(nrow(dataframe1), sampling_number, replace = TRUE), ]
 dataframe2<- dataframe2[sample(nrow(dataframe2), sampling_number, replace = TRUE), ]
 dataframe3<- dataframe3[sample(nrow(dataframe3), sampling_number, replace = TRUE), ]
 p<- ggplot() + 
    geom_density(data = gather(dataframe1), aes(value), color='green', fill= 'green', adjust = 1.5, alpha = 0.1, lwd= 1.5)+ 
    facet_wrap(~key, scales ='free', ncol = 4)+
    xlab(xlab)+
    geom_density(data = gather(dataframe2), aes(value), color='red', fill= 'red', adjust = 1.5, alpha = 0.1, lwd= 1.5)+
    facet_wrap(~key, scales = 'free', ncol = 4)+
    xlab(xlab)+
    geom_density(data = gather(dataframe3), aes(value), color='blue', fill= 'blue', adjust = 1.5, alpha = 0.1, lwd= 1.5)+
    facet_wrap(~key, scales = 'free', ncol = 4)+
    xlab(xlab)+
    ylab("Frequency")+
    ggplot2::theme(plot.title = element_text( size=16, face="bold.italic", hjust = 0.5),
                   axis.text = ggplot2::element_text(face="bold", colour="black", size=12))+
    ggtitle(main_title)
  return(p)  
}


histogram_of_each_column_for_two_dataframes<- function(df1, df2, title, xlab, bin, sampling_number, main_title){
   #' given dataframe
   #' given title name
   #' given xlab value
   #' give bin size
   #' output histogram of all columns
 dataframe1 <- df1[sample(nrow(df1), sampling_number, replace = TRUE), ]
 dataframe2<- df2[sample(nrow(df2), sampling_number, replace = TRUE), ]
 p<- ggplot() + 
    geom_density(data = gather(dataframe1), aes(value), color='green', fill= 'green', adjust = 1.5, alpha = 0.1, lwd= 1.5)+ 
    facet_wrap(~key, scales ='free', ncol = 4)+
    xlab(xlab)+
    geom_density(data = gather(dataframe2), aes(value), color='blue', fill= 'blue', adjust = 1.5, alpha = 0.1, lwd= 1.5)+
    facet_wrap(~key, scales = 'free', ncol = 4)+
    xlab(xlab)+
    ylab("Frequency")+
    ggplot2::theme(plot.title = element_text( size=16, face="bold.italic", hjust = 0.5),
                   axis.text = ggplot2::element_text(face="bold", colour="black", size=12))+
    ggtitle(main_title)+
    ggplot2::theme(panel.background = element_rect(fill = "#BFD5E3"))
  return(p)  
}


tag_facet <- function(p, open = "(", close = ")", tag_pool = letters, x = -Inf, y = Inf, 
                      hjust = -0.5, vjust = 1.5, fontface = 2, family = "", colour, ...) {
  
  gb <- ggplot_build(p)
  lay <- gb$layout$layout
  tags <- cbind(lay, label = paste0(open, tag_pool[lay$PANEL], close), x = x, y = y)
  p + 
    geom_text(data = tags, aes_string(x = "x", y = "y", label = "label"), colour=colour, ..., hjust = hjust, 
                vjust = vjust, fontface = fontface, family = family, inherit.aes = FALSE)+
    scale_colour_manual(values= colour)+
    ggplot2::theme(panel.background = ggplot2::element_rect(fill = "white", colour = "black")) +
      ggplot2::theme(axis.title.x = ggplot2::element_text(face = "bold", colour = "black", size = 20)) +
      ggplot2::theme(axis.text = ggplot2::element_text(face = "bold", colour = "black", size = 20)) +
      ggplot2::theme(axis.title.y = ggplot2::element_text(face = "bold", colour = "black", size = 20)) +
      ggplot2::theme(panel.border = ggplot2::element_rect(colour = "black", fill = NA, size = 1))+
    ggplot2::theme(plot.title = element_text(size = 20))+
    ggplot2::theme(text = element_text(size = 20))
}

In [ ]:
needed_antibody<- c('141Pr_CD196','142Nd_CD19', '143Nd_CD5',  '144Nd_CD38', '146Nd_IgD','147Sm_CD11c', '148Nd_CD16',
                    '149Sm_CCR4', '150Nd_CD43', '151Eu_CD69', '152Sm_CD21', '153Eu_CXCR5', '154Sm_CD62L', '158Gd_CD27',
                     '159Tb_CD22', '160Gd_CD14', '163Dy_CXCR3', '164Dy_CD23', '166Er_CD24', '167Er_CCR7',  '171Yb_CD20',
                    '172Yb_IgM', '173Yb_HLA-DR', '174Yb_CD49d', '175Lu_CXCR4', '176Yb_CD56')
needed_antibody1<- c('141Pr_CD196','142Nd_CD19', '143Nd_CD5',  '144Nd_CD38', '146Nd_IgD','147Sm_CD11c', '150Nd_CD43', '151Eu_CD69', '152Sm_CD21', '153Eu_CXCR5', '154Sm_CD62L', '158Gd_CD27',
                     '159Tb_CD22',  '163Dy_CXCR3', '164Dy_CD23', '166Er_CD24', '167Er_CCR7',  '171Yb_CD20',
                    '172Yb_IgM', '173Yb_HLA-DR', '174Yb_CD49d', '175Lu_CXCR4')

In [ ]:
need = needed_antibody1

In [ ]:
path = "D:/AAAAAA_PhD_data/"
batch <- 6
anchor <- "anchorstim"
nonanchor <- "nonanchor_vali"

In [ ]:
##compare anchors of different batches
# rans1<- data.frame()
# for (i in c(2,3,5,6)){
path_needed1<- glue("{path}Data/cytofruv_normalized_one_batch_at_a_time/Batch_{batch}_1_{anchor}.fcs")
path_needed2<- glue("{path}Data/cytofruv_normalized_one_batch_at_a_time/Batch_{batch}_{batch}_{anchor}.fcs")
# dataframe_res1<- fcs_to_dataframe(path_needed1, 
#                 need,
#                 FALSE)
# dataframe_res2<- fcs_to_dataframe(path_needed2, 
#                 need,
#                 FALSE)
# ans1<- emd_for_each_marker_for_two_fcs(path_needed1, path_needed2, TRUE, 1000, need)
# rans1<- rbind(rans1, ans1)
#     }

In [ ]:
dataframe_res1<- fcs_to_dataframe(path_needed1, 
                need,
                FALSE)
dataframe_res2<- fcs_to_dataframe(path_needed2, 
                need,
                FALSE)

In [ ]:
head(dataframe_res1)

In [ ]:
ans1<- emd_for_each_marker_for_two_fcs(path_needed1, path_needed2, TRUE, 500, need)

In [ ]:
ans1

In [ ]:
#####get the difference before and after correction for a method1 
####get the difference before and after correction for another method2

In [ ]:
path = "D:/AAAAAA_PhD_data/"
batch <- 2
anchor <- "anchorstim"
nonanchor <- "nonanchor_vali"
samplename <- 3640

In [ ]:
append_suffix <- function(number) {
  last_digit <- number %% 10
  suffix <- switch(last_digit,
                   "1" = "st",
                   "2" = "nd",
                   "3" = "rd",
                   "th")
  if (last_digit >= 4 | last_digit == 0) {
    suffix <- "th"
  }
  return(paste0(number, suffix))
}

In [ ]:
### this is before correction example both
path_needed1<- output_path(samplename, glue("{path}Data/experiment_UMvsM/"), "1st")
path_needed2<- output_path(samplename, glue("{path}Data/experiment_UMvsM/"), glue("{append_suffix(batch)}"))

In [ ]:
##this is for after correction example for cytofruv
path_needed3<- glue("{path}Data/cytofruv_normalized_one_batch_at_a_time/Batch_{batch}_1_{anchor}.fcs")
path_needed4<- glue("{path}Data/cytofruv_normalized_one_batch_at_a_time/Batch_{batch}_{batch}_{anchor}.fcs")

In [ ]:
##this is for after correction example for cytonorm
filenametocode <- file.path("..", "R3_ML_mut_vs_unmut_scripts", "filenametocode.csv")
filenametocodedata <- read.csv(filenametocode)
samplenamerows <- filenametocodedata[filenametocodedata$name == samplename, ]
batchelement <- samplenamerows$code[grep(glue("Batch_{batch}"), samplenamerows$code)]
path_needed5<- output_path(samplename, glue("{path}Data/experiment_UMvsM/"), "1st")
path_needed6<- glue("{path}Data/all_cytonorm_normalized/{batchelement}")

In [ ]:
dataframe_res1<- fcs_to_dataframe(path_needed1, 
                need,
                FALSE)
dataframe_res2<- fcs_to_dataframe(path_needed2, 
                need,
                FALSE)
dataframe_res3<- fcs_to_dataframe(path_needed3, 
                need,
                FALSE)
dataframe_res4<- fcs_to_dataframe(path_needed4, 
                need,
                FALSE)
dataframe_res5<- fcs_to_dataframe(path_needed5, 
                need,
                FALSE)
dataframe_res6<- fcs_to_dataframe(path_needed6, 
                need,
                FALSE)

In [ ]:
ans1<- emd_for_each_marker_for_two_fcs(path_needed1, path_needed2, TRUE, 500, need)
ans2_method1<- emd_for_each_marker_for_two_fcs(path_needed3, path_needed4, TRUE, 500, need)   ##cytofruv
ans3_method2<- emd_for_each_marker_for_two_fcs(path_needed5, path_needed6, TRUE, 500, need)   ##cytonorm

In [ ]:
ans3_method2

In [ ]:
ans1

In [ ]:
t_test_result <- t.test(ans3_method2$distance, ans2_method1$distance, paired=TRUE)

# Print the result
print(t_test_result)
######mean difference means (ans2_method1$distance - ans3_method2$distance)

In [ ]:
#######################################################################
######################################################################

In [ ]:
technique<- "CytofRUV"
sample_number<- glue("{samplename} (Validation)")
first_batch<- "one"
second_batch<- glue("{batch}")

main_title<- paste0(
                    "Overlaid Density Plots of sample ",
                    sample_number,
                    " from batch ",
                    first_batch,
                    " (green) ",
                  
                    "and batch ",
                    second_batch,
                    " (blue) ",
                    " after ",
                     technique,
                     "-correction"
                    )
p<- histogram_of_each_column_for_two_dataframes(dataframe_res3, dataframe_res4, "markers expression", "markers expressions", 100, 2000, main_title) ###for scaled dataframe  

p<- tag_facet(p, 
              open = "emd ",
              close = " ",
          tag_pool = round(ans1$distance, digit = 3),
             colour = "blue",
             hjust = -4.0, vjust = 2.5,
             fontface = 20)

options(repr.plot.width=20, repr.plot.height=15)



In [ ]:
p

In [ ]:
technique<- "ML"
sample_number<- glue("{samplename} (Validation)")
first_batch<- "one"
second_batch<- glue("{batch}")

main_title<- paste0(
                    "Overlaid Density Plots of sample ",
                    sample_number,
                    " from batch ",
                    first_batch,
                    " (green) ",
                  
                    "and batch ",
                    second_batch,
                    " (blue) ",
                    " after ",
                     technique,
                     "-correction"
                    )
p<- histogram_of_each_column_for_two_dataframes(dataframe_res5, dataframe_res6, "markers expression", "markers expressions", 100, 2000, main_title) ###for scaled dataframe  

p<- tag_facet(p, 
              open = "emd ",
              close = " ",
          tag_pool = round(ans1$distance, digit = 3),
             colour = "blue",
             hjust = -4.0, vjust = 2.5,
             fontface = 20)

options(repr.plot.width=20, repr.plot.height=15)



In [ ]:
p

In [ ]:
#########################
########################
########################
########################

In [ ]:
#############

In [ ]:
path_needed1<- output_path("3620", glue("{path}Data/experiment_UMvsM/"), "1st")
path_needed2<- output_path("3620", glue("{path}Data/experiment_UMvsM/"), "6th")
path_needed3<- glue("{path}Data/all_cytonorm_normalized/Batch_6_19nonanchor1.fcs")

In [ ]:
dataframe_res1<- fcs_to_dataframe(path_needed1, 
                need,
                FALSE)
dataframe_res2<- fcs_to_dataframe(path_needed2, 
                need,
                FALSE)
dataframe_res3<- fcs_to_dataframe(path_needed3, 
                need,
                FALSE)


In [ ]:
cyto_ans1<- emd_for_each_marker_for_two_fcs(path_needed1, path_needed2, TRUE, 1500, need)
#cyto_ans2<- emd_for_each_marker_for_two_fcs(path_needed1, path_needed3, TRUE, 1500, need)

In [ ]:
cyto_ans1

In [ ]:
technique<- "ML-based"
sample_number<- "3620"
first_batch<- "one"
second_batch<- "six"

main_title<- paste0(
                    "Overlaid histograms of sample \n ",
                    sample_number,
                    " from batch ",
                    first_batch,
                    " (green) ",
                  
                    "and batch ",
                    second_batch,
                      "\n ",
                     "before (red) and after (blue) \n",
                      technique,
                     "-correction"
                    )
main_title <- glue("Overlaid Density Plots of sample {sample_number} from batch {first_batch} (green) and batch {second_batch} (blue) after ML based correction")                

# 2 histogram first

In [ ]:
p<- histogram_of_each_column_for_two_dataframes(dataframe_res1, dataframe_res2, "markers expression", "markers expressions", 100, 2000, main_title) ###for scaled dataframe

In [ ]:
p<- tag_facet(p, 
              open = "emd ",
          tag_pool = round(cyto_ans1$distance, digit = 3),
             colour = "blue",
             hjust = -4.0, vjust = 2.5,
             fontface = 4)

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)

In [ ]:
p

In [ ]:
p<- histogram_of_each_column_for_three_dataframes(dataframe_res1, dataframe_res2,dataframe_res3, "markers expression", "markers expressions", 100, 2000, main_title) ###for scaled dataframe

In [ ]:
p<- tag_facet(p, 
              open = "emd ",
          tag_pool = round(ans2$distance, digit = 3),
             colour = "blue",
             hjust = -4.5, vjust = 2.5,
             fontface = 4)
p<- tag_facet(p, 
              open = "emd ",
          tag_pool = round(ans1$distance, digit = 3),
             colour = "red",
             hjust = -2.5, vjust = 2.5,
             fontface = 4)

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)

In [ ]:
p